# Final Presentation for the Transformers team



In [ ]:
?????

Hello world!


## Assignment 1
In Assignment 1 we are building a simple graph processing class. We build an undirected graph and implement two functionalties:

- Find downstream vertices
- Find alternative edges

## Making a graph

- A class for processing undirected graphs.  
- This class provides functionalities: initializing a graph, finding downstream vertices of an edge, and identifying alternative edges for ensuring graph connectivity.

### Attributes:
- **graph**: A NetworkX graph representing the processed graph.


### Arguments
- `vertex_ids` (List[int]): List of vertex IDs.
- `edge_ids` (List[int]): List of edge IDs.
- `edge_vertex_id_pairs` (List[Tuple[int, int]]): List of tuples - - representing vertex pairs for each edge.
- `edge_enabled` (List[bool]): List indicating whether each edge is either enabled (true) or disabled (false) 
- `source_vertex_id` (int): ID of the source vertex.


## Module structure

The module `model_processor` is made out of 5 functions:
1. `load_input_data`  
    Reads the data from the three files and checks that it is correct.
   **Inputs**:  
   - `active_data_path` (str): Path to active power data (parquet)  
   - `reactive_data_path` (str): Path to reactive power data (parquet)  
   - `model_data_path` (str): Path to static network model (JSON)  
   **Outputs**:  
   - `active_df` (DataFrame), `reactive_df` (DataFrame), `dataset` (dict)

2. `run_updated_power_flow_analysis`  
    Takes the data that was read from `load_input_data` and performs a PGM batch update.
   **Inputs**:  
   - `active_df` (DataFrame): Active power values indexed by time  
   - `reactive_df` (DataFrame): Reactive power values indexed by time  
   - `dataset` (dict): Static model of the power grid  
   **Output**:  
   - `output` (dict): Power flow simulation results

3. `node_voltage_summary`  
    Takes as input the model from `run_updated_power_flow_analysis`. Aggregates voltage results and creates a dataframe for them.
   **Inputs**:  
   - `output` (dict): Simulation output  
   - `timestamps` (Index): Time index from the input data  
   **Output**:  
   - `node_df` (DataFrame): Voltage stats per timestamp and node

4. `line_statistics_summary`  
   Takes as input the model from `run_updated_power_flow_analysis`. Aggregates line results and creates a dataframe for them/
   **Inputs**:  
   - `output` (dict): Simulation output  
   - `timestamps` (Index): Time index from the input data  
   **Output**:  
   - `line_df` (DataFrame): Energy loss and loading stats per line

5. `data_processing`  
   Uses all the functions in order and returns both dataframes. 
   **Inputs**:  
   - `active_data_path`, `reactive_data_path`, `model_data_path` (str): Input file paths  
   **Outputs**:  
   - `node_df`, `line_df` (DataFrames): Summaries of node voltages and line metrics

### Raises

- **`IDNotUniqueError`**  
  Raised if there are duplicate vertex or edge IDs.

- **`InputLengthDoesNotMatchError`**  
  Raised if the length of `edge_enabled` does not match the lengths of the input lists `edge_ids`.

- **`IDNotFoundError`**  
  Raised if a vertex ID in `edge_vertex_id_pairs` does not exist in `vertex_ids`.

- **`GraphNotFullyConnectedError`**  
  Raised if the graph is not fully connected.

- **`GraphCycleError`**  
  Raised if the graph contains one or more cycles.


The functionalities of this class can be tested through a grid, which is created by using the code below: 

In [ ]:
vertex_ids = [0, 2, 4, 6, 10]
edge_ids = [1, 3, 5, 7, 8, 9]
edge_vertex_id_pairs = [(0, 2), (0, 4), (0, 6), (2, 4), (4, 6), (2, 10)]
edge_enabled = [True, True, True, False, False, True]
source_vertex_id = 10

grid = GraphProcessor(vertex_ids, edge_ids, edge_vertex_id_pairs, edge_enabled, source_vertex_id)

### Graph depiction 
The graph created above can be visualized in the following way:

### Finding downstream vertices 

**Arguments:**
- `starting_edge_id (int)`: ID of the edge to find downstream vertices for.

**Returns:**
- `List[int]`: List of vertex IDs downstream from the specified edge.

**Method:**
1. **Check edge existence**:
   - Raise an exception if the ID is not in the edge list.

2. **Check if the edge is enabled**:
   - If disabled, there's no downstream effect → return empty list.

3. **Get the edge’s vertex pair**:
   - Use `edge_vertex_id_pairs` to get `(vertex1, vertex2)`.

4. **Simulate edge removal**:
   - Copy the graph to avoid modifying the original.
   - Remove the given edge from the copied graph.

5. **Compute connected components**:
   - Use NetworkX to find components after edge removal.

6. **Find source-containing component**:
   - Identify the group containing `source_vertex_id`.

7. **Return the disconnected part**:
   - Return the other component (not containing the source).
   - If no separation occurred, return an empty list.

In [ ]:
downstream_vertices1 = graph.find_downstream_vertices(1) 
downstream_vertices1


NameError: name 'graph' is not defined

### Finding alternative edges 

Find alternative disabled edges that, if re-enabled, can restore full connectivity to the graph without introducing a cycle — after a specific edge is disabled.

### Arguments:
- `disabled_edge_id (int)`: ID of the edge that is about to be disabled.

### Returns:
- `List[int]`: List of edge IDs that could be re-enabled as alternatives to restore full graph connectivity without forming a cycle.



   

### Method:

1. **Validate the input edge**:
   - If the given edge ID doesn't exist, raise `IDNotFoundError`.
   - If the edge is already disabled, raise `EdgeAlreadyDisabledError`.

2. **Find all currently disabled edges**:
   - Loop through `edge_enabled` and collect IDs of disabled edges.

3. **Prepare a baseline edge status map**:
   - Copy the current state of all edges.
   - Mark the given `disabled_edge_id` as `False` (simulate disabling it).

4. **Test each disabled edge**:
   - For each `test_edge_id` in the disabled list:
     - Temporarily enable just that edge in a new status map.
     - Reconstruct the graph using this temporary configuration.

5. **Check the test graph**:
   - If the resulting graph is **connected** AND **does not form a cycle**, then it's a valid alternative.

6. **Add valid alternatives to the list**:
   - If both conditions are met, add `test_edge_id` to `alt_list`.

7. **Return the list** of alternative edge IDs.


In [ ]:
alternative_edges = grid.find_alternative_edges(3)
alternative_edges

# Assignment2 - TO DO 

### Finding an optimal tap position
 Finding the optimal tap position of the transformer based on one of the following criteria:
- The minimal total energy loss of all the lines and the whole time period.
- The minimal (averaged across all nodes) deviation of (max and min) p.u. node voltages with respect to 1 p.u.


###  Arguments

- **`input_network_data`** (`.json`):  
  JSON file describing the electrical network topology and components.

- **`active_power_profile_path`** (`.parquet`):  
  File path to the active power profile dataset.

- **`reactive_power_profile_path`** (`.parquet`):  
  File path to the reactive power profile dataset.

- **`optimize_by`** (`int`):  
  Optimization objective:
  - `0`: Minimize power losses  
  - `1`: Minimize voltage deviation


# METHOD: TO DO + DEMO

# METHOD: TO DO

# N1 - TO DO